<b>Generating data for Algorithm Project</b>

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import glob
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import string

files1 = glob.glob('sraa/realaviation/*')
files2 = glob.glob('sraa/simaviation/*')
files3 = glob.glob('sraa/realauto/*')
files4 = glob.glob('sraa/simauto/*')


utf8 = ['\x83', '\x93', '\x97', '\xa3', '\xa7', '\xab', '\xaf', '\xb3', '\xb7', '\xbb', '\xbf', '\xc3', 
        '\xc7', '\xcb', '\xcf', '\xd3', '\xd7', '\xdb', '\xdf', '\xe3', '\xe7', '\xeb', '\xef', '\xf3', 
        '\xf7', '\xfb', '\x84', '\x88', '\x90', '\x94', '\x9c', '\xa0', '\xa4', '\xa8', '\xac', '\xb0', 
        '\xb4', '\xb8', '\xbc', '\xc0', '\xc4', '\xc8', '\xcc', '\xd0', '\xd4', '\xd8', '\xdc', '\xe0', 
        '\xe4', '\xe8', '\xec', '\xf0', '\xf4', '\xf8', '\xfc', '\x85', '\x91', '\x95', '\x99', '\xa1', 
        '\xa5', '\xa9', '\xad', '\xb1', '\xb5', '\xb9', '\xbd', '\xc1', '\xc5', '\xc9', '\xcd', '\xd1', 
        '\xd5', '\xd9', '\xdd', '\xe1', '\xe5', '\xe9', '\xed', '\xf1', '\xf5', '\xf9', '\xfd', '\x92', 
        '\x96', '\xa2', '\xa6', '\xaa', '\xae', '\xb2', '\xb6', '\xba', '\xbe', '\xc2', '\xc6', '\xca', '\xce', 
        '\xd2', '\xd6', '\xda', '\xde', '\xe2', '\xe6', '\xea', '\xee', '\xf2', '\xf6', '\xfa', '\xfe']

directory = glob.glob('sraa/*')
stemmer = PorterStemmer()


        

In [3]:
documents = []

for folder in directory:
    print folder
    tmp = []
    files = glob.glob(folder+"/*")
    nums = 0
    for fileName in files:
        f = open(fileName,'r')
        data = f.read()
        data = ''.join(i if ord(i)<128 else ' ' for i in data )
        data = data.lower()
        
        #toker = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)
        #dataA = toker.tokenize(data)
        
        dataA = word_tokenize(data)
        dataA_without_punc = [i for i in dataA if i not in string.punctuation]
        
        single = ' '.join([stemmer.stem(w) for w in dataA_without_punc])
        tmp.append(single)
    documents.append(tmp)

sraa\realauto
sraa\realaviation
sraa\simauto
sraa\simaviation


In [4]:
from nltk.corpus import stopwords
from collections import defaultdict
from gensim import corpora,models, similarities

def preprocess(documents):
    # documents = ['babak is khar','hasan is good']
    stopList = stopwords.words('english')
    print len(stopList)
    texts = [[word for word in document.lower().split() if word not in stopList]
         for document in documents]
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1

    return [[token for token in text if frequency[token] > 1]
        for text in texts]
    
    
    
def makeCorpus(texts):
    # texts = [['babak', 'is' , 'khar'],['hasan', 'is','good']]
    # it means we do some preprocess like: stopword, common word, ...
    dictionary = corpora.Dictionary(texts)
#     print dictionary.token2id
    return [dictionary.doc2bow(text) for text in texts]

In [5]:
alldocs = []
for docs in documents:
    for doc in docs:
        alldocs.append(doc)

        
texts = preprocess(alldocs)
corpus = makeCorpus(texts)

127


In [7]:
print len(corpus[0])

63


In [9]:
print corpus[10]

[(0, 1), (2, 4), (9, 1), (10, 1), (11, 3), (12, 1), (13, 1), (14, 1), (16, 4), (18, 1), (23, 1), (28, 1), (38, 1), (44, 1), (45, 1), (46, 2), (47, 1), (50, 1), (51, 1), (53, 4), (54, 4), (58, 2), (60, 1), (81, 2), (100, 1), (114, 1), (121, 2), (139, 9), (144, 1), (149, 1), (152, 2), (155, 1), (163, 1), (164, 1), (165, 1), (166, 5), (167, 1), (183, 1), (185, 1), (187, 2), (191, 4), (193, 2), (205, 1), (224, 1), (236, 1), (259, 4), (270, 1), (289, 1), (314, 1), (315, 1), (318, 1), (319, 1), (320, 1), (323, 2), (324, 1), (325, 1), (329, 1), (332, 1), (337, 1), (340, 1), (345, 1), (347, 1), (348, 1), (349, 1), (350, 1), (351, 1), (352, 1), (353, 1), (354, 1), (356, 2), (362, 3), (363, 1), (365, 1), (367, 1), (370, 1), (371, 1), (374, 1), (375, 4), (376, 1), (378, 1), (379, 1), (381, 1), (385, 1), (386, 1), (387, 2), (388, 2), (390, 1), (394, 1), (395, 1), (401, 1), (404, 1), (416, 1), (515, 1), (517, 1), (528, 1), (539, 2), (553, 2), (555, 1), (556, 2), (558, 1), (559, 1), (560, 1), (561, 

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords

# train_set = (documents) #Documents
train_set = (alldocs) #Documents

test_set = (["Content-Type text 7bit news."]) #Query
stopWords = stopwords.words('english')

vectorizer = CountVectorizer(stop_words = stopWords)
transformer = TfidfTransformer()

trainVectorizerArray = vectorizer.fit_transform(train_set).toarray()
testVectorizerArray = vectorizer.transform(test_set).toarray()
print 'Fit Vectorizer to train set', trainVectorizerArray
print 'Transform Vectorizer to test set', testVectorizerArray

transformer.fit(trainVectorizerArray)
print transformer.transform(trainVectorizerArray).toarray()

#transformer.fit(testVectorizerArray)

#tfidf = transformer.transform(testVectorizerArray)
#print 'dens

#e'
#print tfidf.todense()

AttributeError: 'list' object has no attribute 'lower'

In [21]:
from nltk.corpus import stopwords
stopWords = stopwords.words(['english'])

In [27]:
print documents[24]

Path: news.jprc.com!newsfeed.sgi.net!uwm.edu!math.ohio-state.edu!howland.erols.net!wn3feed!worldnet.att.net!135.173.83.225!attworldnet!newsadm
From: news@4x4review.com (4X4REVIEW)
Newsgroups: rec.autos.misc
Subject: !!!!!!NEW WEBSITE 4x4/SUV product and vehicle reviews and MUCH MORE!!!!!!!!!!!!
Date: 24 Sep 1998 04:03:13 GMT
Organization: AT&T WorldNet Services
Lines: 9
Message-ID: <3609c46d.95136777@netnews.worldnet.att.net>
NNTP-Posting-Host: 12.64.40.208
X-Newsreader: Forte Free Agent 1.11/32.235
Xref: news.jprc.com rec.autos.misc:22958

http://www.4x4review.com/ng
There's a new web site on the net that does 4x4 and SUV product and
vehicle reviews.  Next month, be the first to see the review and
technical data that goes with Detroit's new GEARLESS LOCKER!
In addition, they are doing bi-weekly newsletters, selling books and
videos, listing 4x4 clubs all around the world, they have discussion
forums and much much more!  Take a peek and see what's new at:
http://www.4x4review.com/ng




In [32]:
doc = '''
Path: news.jprc.com!newsfeed.sgi.net!uwm.edu!math.ohio-state.edu!
howland.erols.net!wn3feed!worldnet.att.net!135.173.83.225!attworldnet!
newsadm\nFrom: news@4x4review.com (4X4REVIEW)\nNewsgroups: 
rec.autos.misc\nSubject: !!!!!!NEW WEBSITE 4x4/SUV product and vehicle reviews and MUCH MORE!!!!!!!!!!!!\n
Date: 24 Sep 1998 04:03:13 GMT\nOrganization: AT&T WorldNet Services\nLines: 9\nMessage-ID:
\nNNTP-Posting-Host: 12.64.40.208\nX-Newsreader: Forte Free Agent 1.11/32.235\nXref: 
news.jprc.com rec.autos.misc:22958\n\nhttp://www.4x4review.com/ng\nThere's a new web site on the net that does 4x4 and SUV 
product and\nvehicle reviews.  Next month, be the first to see the review and\ntechnical data that goes with Detroit\x92s 
new GEARLESS LOCKER!\nIn addition, they are doing bi-weekly newsletters, selling books and\nvideos, listing 4x4 clubs all around the world, they have discussion\nforums and much much more!  Take a peek and see what's new at:\nhttp://www.4x4review.com/ng\n\n
'''

In [33]:
print doc


Path: news.jprc.com!newsfeed.sgi.net!uwm.edu!math.ohio-state.edu!
howland.erols.net!wn3feed!worldnet.att.net!135.173.83.225!attworldnet!
newsadm
From: news@4x4review.com (4X4REVIEW)
Newsgroups: 
rec.autos.misc
Subject: !!!!!!NEW WEBSITE 4x4/SUV product and vehicle reviews and MUCH MORE!!!!!!!!!!!!

Date: 24 Sep 1998 04:03:13 GMT
Organization: AT&T WorldNet Services
Lines: 9
Message-ID:

NNTP-Posting-Host: 12.64.40.208
X-Newsreader: Forte Free Agent 1.11/32.235
Xref: 
news.jprc.com rec.autos.misc:22958

http://www.4x4review.com/ng
There's a new web site on the net that does 4x4 and SUV 
product and
vehicle reviews.  Next month, be the first to see the review and
technical data that goes with Detroit\x92s 
new GEARLESS LOCKER!
In addition, they are doing bi-weekly newsletters, selling books and
videos, listing 4x4 clubs all around the world, they have discussion
forums and much much more!  Take a peek and see what's new at:
http://www.4x4review.com/ng





In [ ]:
name = 'sraa/realauto/

In [14]:
from gensim import corpora,models,similarities
newDoc = []
for d in documents:
    for i in d:
        newDoc.append(i)
print len(newDoc)

73218


In [ ]:
model = models.Word2Vec(newDoc,size=100,window=5,min_count=5,workers=4)
model.save("word.txt")